<a href="https://colab.research.google.com/github/dafnep4/Dafne-Petrelli-s-coding/blob/main/julia_pysr_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## installation of Julia

!curl -fsSL https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.3-linux-x86_64.tar.gz -o julia.tar.gz
!tar -xvzf julia.tar.gz
!mv julia-1.9.3 /usr/local/julia

import os
os.environ["PATH"] = "/usr/local/julia/bin:" + os.environ["PATH"]
!pip install julia pysr

import julia
julia.install()  # Set up PyCall

from julia import Main
Main.eval('using Pkg')
Main.eval('Pkg.activate("/content/julia_env")')
Main.eval('Pkg.add(url="https://github.com/MilesCranmer/SymbolicRegression.jl")')
Main.eval('Pkg.add("Symbolics")')
Main.eval('Pkg.add("SymbolicUtils")')

julia-1.9.3/
julia-1.9.3/lib/
julia-1.9.3/lib/libjulia.so.1.9
julia-1.9.3/lib/libjulia.so
julia-1.9.3/lib/libjulia.so.1
julia-1.9.3/lib/julia/
julia-1.9.3/lib/julia/libcholmod.so.3.0.14
julia-1.9.3/lib/julia/libuv.so.2
julia-1.9.3/lib/julia/libuv.so.2.0.0
julia-1.9.3/lib/julia/libopenlibm.so.4.0
julia-1.9.3/lib/julia/libpcre2-8.so.0
julia-1.9.3/lib/julia/libstdc++.so.6
julia-1.9.3/lib/julia/libunwind.so
julia-1.9.3/lib/julia/libccalltest.so.debug
julia-1.9.3/lib/julia/libnghttp2.so.14
julia-1.9.3/lib/julia/libatomic.so.1.2.0
julia-1.9.3/lib/julia/libopenlibm.so.4
julia-1.9.3/lib/julia/libquadmath.so
julia-1.9.3/lib/julia/libklu.so.1
julia-1.9.3/lib/julia/libgfortran.so.5.0.0
julia-1.9.3/lib/julia/librbio.so
julia-1.9.3/lib/julia/libmbedcrypto.so
julia-1.9.3/lib/julia/libccolamd.so.2.9.6
julia-1.9.3/lib/julia/libsuitesparseconfig.so.5.10.1
julia-1.9.3/lib/julia/libssp.so.0
julia-1.9.3/lib/julia/libopenblas64_.so.0
julia-1.9.3/lib/julia/sys.so
julia-1.9.3/lib/julia/libopenlibm.so
julia-1

JuliaError: Exception 'Refusing to add package `SymbolicRegression [8254be44]`.
Package `SymbolicRegression=3e3f1a6c-6ca0-5d76-8f51-48f1c1eaf9e9` with the same name already exists as a direct dependency.
To remove the existing package, use `import Pkg; Pkg.rm("SymbolicRegression")`.
' occurred while calling julia code:
Pkg.add(url="https://github.com/MilesCranmer/SymbolicRegression.jl")

In [ ]:

with open("/content/julia_env/src/MyFitness.jl", "w") as f:
    f.write("""
module MyFitness

using SymbolicRegression, Symbolics, SymbolicUtils


function observables(potential, phi)
    @variables phi
    V_phi_expr = potential(phi)
    V_first = SymbolicUtils.derivative(V_phi_expr, phi)
    V_second = SymbolicUtils.derivative(V_first, phi)

    V = eval(build_function(V_phi_expr, phi)[2])
    dV = eval(build_function(V_first, phi)[2])
    dV2 = eval(build_function(V_second, phi)[2])

    Vp = V(phi)
    Vp1 = dV(phi)
    Vp2 = dV2(phi)


  # here I am computing the values for the slow-roll parameters which are based on the derivatives computed above.
  # these will be used to obtain the different parameters to study the accuracy of the potential reconstructed.
  epsilon = 1/2 * ((Vp1) / (Vp))^2
  eta = (Vp2) / (Vp)

  # finding the value for n_s, which corresponds to the spectral index.
  # and finding the value for r, which corresponds to the tensor-to-scalar ratio.
  n_s = 1 - (6 * epsilon) + (2 * eta)
  r = 16 * epsilon

  return n_s, r

end

function fitness(potential)
    try
        n_s, r = observables(potential, 5.5)
        return (n_s - 0.965)^2 + (r - 0.04)^2
    catch
        return Inf
    end
end
export fitness

end # module
""")



In [ ]:
import sympy as sp
import numpy as np
from pysr import PySRRegressor

phi = sp.Symbol("phi")
X = np.linspace(0.01, 6.0, 200).reshape(-1, 1)
Y = np.zeros_like(X)

model = PySRRegressor(
    niterations=200,
    binary_operators=["+", "-", "*", "^"],
    unary_operators=["exp"],
    model_selection="best",
    maxsize=20,
    julia_project="/content/julia_env",  # points to your environment folder
    loss="MyFitness.fitness",            # reference to your custom fitness function
    constraints={"^": (1, 3)},
    random_state=0,
    parallelism="serial",
    deterministic=True,
    verbosity=1,
)

model.fit(X, Y)

best_potential = model.get_best()["sympy_format"]
print("Best inflationary potential:")
print(best_potential)

V_phi_func = sp.lambdify(phi, best_potential, "numpy")
